In [60]:
import random
import pylab

In [61]:
class Location(object):
    
    def __init__(self,x,y):
        self.x, self.y = x, y
        
    def move(self, deltaX, deltaY):
        return Location(self.x + deltaX, self.y + deltaY)
    
    def getX(self):
        return self.x
    
    def getY(self):
        return self.y

    def distFrom(self, other):
        ox, oy = other.x, other.y 
        xDist, yDist = self.x - ox, self.y - oy
        return (xDist**2 + yDist**2)**0.5 # Euclidean Distance
    
    def __str__(self):
        return '<' + str(self.x) + ', ' + str(self.y) + '>'
    

In [62]:
class Field(object):
    
    def __init__(self):
        self.drunks = {}
        
    def addDrunk(self, drunk, loc):
        if drunk in self.drunks:
            raise ValueError('Duplicate drunk')
        else:
            self.drunks[drunk] = loc
            
    def moveDrunk(self, drunk):
        if drunk not in self.drunks:
            raise ValueError('Drunk not in field')
        xDist, yDist = drunk.takeStep() 
        currentLocation = self.drunks[drunk]
        #use move method of Location to get new location
        self.drunks[drunk] = currentLocation.move(xDist, yDist)
        
    def getLoc(self, drunk):
        if drunk not in self.drunks:
            raise ValueError('Drunk not in field')
        return self.drunks[drunk]

In [63]:
class Drunk(object): 
    def __init__(self, name = None):
        self.name = name
    def __str__(self):
        if self != None:
            return self.name
        return 'Anonymous'    
    

In [64]:
class UsualDrunk(Drunk):
    def takeStep(self):
        stepChoices = [(0,1), (0,-1), (1,0), (-1,0)] # Random walk modelling
        return random.choice(stepChoices)

In [65]:
def walk(f, d, numSteps): # f = Field() , d = Drunk()
    start = f.getLoc(d)
    for s in range(numSteps):
        f.moveDrunk(d)
    return start.distFrom(f.getLoc(d))

def simWalks(numSteps, numTrials, dClass): # dClass = Drunk Class
    Homer = dClass()
    origin = Location(0,0)
    distances = []
    for t in range(numTrials):
        f = Field()
        f.addDrunk(Homer, origin)
        distances.append(round(walk(f, Homer, numSteps), 1)) 
    return distances

def drunkTest(walkLenghts, numTrials, dClass):
    for numSteps in walkLenghts:
        distances = simWalks(numSteps, numTrials, dClass)
        print(dClass.__name__, 'random walk of', numSteps, 'steps')
        print(' Mean =', round(sum(distances)/len(distances),4))
        print(' Max=', max(distances), 'Min=', min(distances))

In [66]:
drunkTest((10, 100, 1000, 10000), 100, UsualDrunk)

UsualDrunk random walk of 10 steps
 Mean = 2.726
 Max= 7.1 Min= 0.0
UsualDrunk random walk of 100 steps
 Mean = 9.695
 Max= 23.5 Min= 0.0
UsualDrunk random walk of 1000 steps
 Mean = 27.811
 Max= 61.1 Min= 3.2
UsualDrunk random walk of 10000 steps
 Mean = 85.1
 Max= 255.0 Min= 6.0


In [67]:
drunkTest((0,1), 100, UsualDrunk)

UsualDrunk random walk of 0 steps
 Mean = 0.0
 Max= 0.0 Min= 0.0
UsualDrunk random walk of 1 steps
 Mean = 1.0
 Max= 1.0 Min= 1.0


In [68]:
class NSDrunk(Drunk):
    def takeStep(self):
        stepChoices = [(0, 1), (0, -2), (1, 0), (-1, 0)]
        return random.choice(stepChoices)
    
class EWDrunk(Drunk):
    def takeStep(self):
        stepChoices = [(1, 0), (-1, 0)]
        return random.choice(stepChoices)

In [69]:
def simAll(drunkKinds, walkLenghts, numTrials):
    for dClass in drunkKinds:
        drunkTest(walkLenghts, numTrials, dClass)

In [70]:
simAll((UsualDrunk, NSDrunk, EWDrunk), (100, 1000), 10)

UsualDrunk random walk of 100 steps
 Mean = 7.73
 Max= 15.8 Min= 2.0
UsualDrunk random walk of 1000 steps
 Mean = 21.48
 Max= 36.2 Min= 8.9
NSDrunk random walk of 100 steps
 Mean = 21.65
 Max= 55.1 Min= 5.8
NSDrunk random walk of 1000 steps
 Mean = 266.3
 Max= 322.2 Min= 202.4
EWDrunk random walk of 100 steps
 Mean = 8.6
 Max= 20.0 Min= 0.0
EWDrunk random walk of 1000 steps
 Mean = 21.4
 Max= 38.0 Min= 4.0


In [71]:
class styleIterator(object):
    def __init__(self, styles):
        self.index = 0
        self.styles = styles
    def nextStyle(self):
        result = self.styles[self.index]
        if self.index == len(self.styles) - 1:
            self.index = 0
        else:
            self.index += 1
        return result  
        

In [72]:
def simDrunk(numTrials, dClass, walkLenghts):
    meanDistances = []
    for numSteps in walkLenghts:
        print('Starting simulation of', numSteps, 'steps')
        trials = simWalks(numSteps, numTrials, dClass)
        mean = sum(trials)/len(trials)
        meanDistances.append(mean)
    return meanDistances

In [73]:
def simAll1(drunkKinds, walkLenghts, numTrials):
    styleChoice = styleIterator(('m-', 'r:', 'k-.')) # different type of line for the graph
    for dClass in drunkKinds:
        curStyle = styleChoice.nextStyle()
        print('Starting simulation of', dClass.__name__)
        means = simDrunk(numTrials, dClass, walkLenghts)
        pylab.plot(walkLenghts, means, curStyle,
                  label = dClass.__name__)
        pylab.title('Mean Distance from Origin ('
                   + str(numTrials) + 'trials)')
        pylab.xlabel('Number of Steps')
        pylab.ylabel('Distance from Origin')
        pylab.legend(loc = 'best')
        pylab.semilogx()
        pylab.semilogy()

Starting simulation of UsualDrunk
Starting simulation of 10 steps
Starting simulation of 100 steps
Starting simulation of 1000 steps
Starting simulation of 10000 steps
Starting simulation of NSDrunk
Starting simulation of 10 steps
Starting simulation of 100 steps
Starting simulation of 1000 steps
Starting simulation of 10000 steps
Starting simulation of EWDrunk
Starting simulation of 10 steps
Starting simulation of 100 steps
Starting simulation of 1000 steps
Starting simulation of 10000 steps


In [74]:
simAll1((UsualDrunk, NSDrunk, EWDrunk), (10,100,1000,10000), 100)

In [81]:
def getFinalLocs(numSteps, numTrials, dClass):
    locs = []
    d = dClass()
    for t in range(numTrials):
        f = Field()
        f.addDrunk(d, Location(0, 0))
        for s in range(numSteps):
            f.moveDrunk(d)
        locs.append(f.getLoc(d))
    return locs

In [84]:
def plotLocs(drunkKinds, numSteps, numTrials):
    styleChoice = styleIterator(('k+', 'r^', 'mo'))
    for dClass in drunkKinds:
        locs = getFinalLocs(numSteps, numTrials, dClass)
        xVals, yVals = [], []
        for loc in locs:
            xVals.append(loc.getX())
            yVals.append(loc.getY())
        meanX = sum(xVals)/len(xVals)
        meanY = sum(yVals)/len(yVals)
        curStyle = styleChoice.nextStyle()
        pylab.plot(xVals, yVals, curStyle,
                  label = dClass.__name__ + 'mean loc. = <'
                  + str(meanX) + ', ' + str(meanY) + '>')
        pylab.title('Location at End of Walks ('
                   + str(numSteps) + 'steps)')
        pylab.xlabel('Steps East/West of Origin')
        pylab.ylabel('Steps North/South of Origin')
        pylab.legend(loc = 'lower left')

In [85]:
plotLocs((UsualDrunk, NSDrunk, EWDrunk), 100, 200)